## Wine Magazine Scraper

Scrape US wine info from reviews sorted by publishing date descending from Wine Magazine website:

[https://www.winemag.com/?s=&drink_type=wine&country=US&pub_date_web=2019,2019-02-01&page=1&sort_by=pub_date_web&sort_dir=desc&search_type=reviews](https://www.winemag.com/?s=&drink_type=wine&country=US&pub_date_web=2019,2019-02-01&page=1&sort_by=pub_date_web&sort_dir=desc&search_type=reviews)
- - -

In [76]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import os
import time

In [77]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [78]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

In [4]:
# Wine Enthusiasts wine reviews website WineMag.com.  Scraping the results of wines from the USA reviewed 
# in February 2019 sorted by review date, most recent first

url='https://www.winemag.com/?s=&drink_type=wine&country=US&pub_date_web=2019,2019-02-01&page=1&sort_by=pub_date_web&sort_dir=desc&search_type=reviews'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# get page range

pagediv = soup.find('div','pagination')
lis = pagediv.find_all('li')
lastli = lis[-1]
lastpage = int(lastli.find('a')["data-page-number"])
print(lastpage)

183


In [6]:
# Grab a list of links for each wine's review page from the results page of WineMag reviews

cpage = 1

wines_url_list=[]

while cpage<=lastpage:

    wines_ls = soup.find_all('a', class_="review-listing row" )

    for link in wines_ls: 
        href=link['href']
        wines_url_list.append(href.strip())
    
    
    cpage+=1
    
    url = "https://www.winemag.com/?s=&drink_type=wine&country=US&pub_date_web=2019,2019-02-01&page="+str(cpage)+"&sort_by=pub_date_web&sort_dir=desc&search_type=reviews"
    browser.visit(url)
    time.sleep(1)
    print(f"page {cpage}")
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
print(f"list length {len(wines_url_list)}, and cpage = {cpage}")

page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
page 25
page 26
page 27
page 28
page 29
page 30
page 31
page 32
page 33
page 34
page 35
page 36
page 37
page 38
page 39
page 40
page 41
page 42
page 43
page 44
page 45
page 46
page 47
page 48
page 49
page 50
page 51
page 52
page 53
page 54
page 55
page 56
page 57
page 58
page 59
page 60
page 61
page 62
page 63
page 64
page 65
page 66
page 67
page 68
page 69
page 70
page 71
page 72
page 73
page 74
page 75
page 76
page 77
page 78
page 79
page 80
page 81
page 82
page 83
page 84
page 85
page 86
page 87
page 88
page 89
page 90
page 91
page 92
page 93
page 94
page 95
page 96
page 97
page 98
page 99
page 100
page 101
page 102
page 103
page 104
page 105
page 106
page 107
page 108
page 109
page 110
page 111
page 112
page 113
page 114
page 115
page 116
page 117
page 118
page 119
page 120
page 121
page 122
page 123
page 124

In [7]:
print(wines_url_list[0])
print(wines_url_list[-1])

https://www.winemag.com/buying-guide/lucia-2017-garys-vineyard-pinot-noir-santa-lucia-highlands/
https://www.winemag.com/buying-guide/casa-larga-2017-dry-riesling-finger-lakes-305403/


In [82]:
# Visit each link in list of URLs and save wine info to df

wine_list=[]

columns = ['title', 'wine', 'vintage', 'vinyard', 'variety', 'avi', 'region', 'state',
           'alcohol_content', 'size', 'winetype', 'price', 'score','dt_published', 'taster']

#wines_df = pd.DataFrame(wine_list, columns = columns)
index = 0

# Follow each link and scrape the wine's review

for link in wines_url_list:
    browser.visit(link)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    wine_elements=[]

    title=soup.find('div', class_="article-title" ).text.split('(')[0]
    score=soup.find('div', class_='rating').text.split()[0]

    primary_info=soup.find('ul', 'primary-info')
    lis = primary_info.find_all('li','row')
    for li in lis:
        labeldiv = li.find('div','info-label')
        label = labeldiv.text.lower()
        infodiv = li.find('div','info')
        info = infodiv.text.strip()
        
        if "price" in label:
            prices=info.split(',')
            price=prices[0]
        elif "designation" in label:
            vinyard=info
        elif "variety" in label:
            variety=info
        elif "appellation" in label:
            appL = info.split(',')
            alen = len(appL)
            # assume country is last
            state = appL[-2]
            region=""
            avi=""
            if alen>2:
                region = appL[-3]
                if alen>3:
                    avi = appL[-4]
        elif "winery" in label:
            wine=info
    
    secondary_info=soup.find('ul', 'secondary-info')
    lis = secondary_info.find_all('li','row')
    for li in lis:
        labeldiv = li.find('div','info-label')
        label = labeldiv.text.lower()
        infodiv = li.find('div','info')
        info = infodiv.text.strip()
        
        if "alcohol" in label:
            alcohol = info
        elif "size" in label:
            size = info
        elif "category" in label:
            winetype = info
        elif "date" in label:
            dt_published = info
            
    taster=soup.find('div', class_='name').text
    checkvin = re.findall('\d\d\d\d', title)
    if (len(checkvin)>0):
        vintage = re.findall('\d\d\d\d', title)[0]
    else:
        vintage = 'NV'

    wines_df.at[index, "title"] = title
    wines_df.at[index, "wine"] = wine
    wines_df.at[index, "vintage"] = vintage    
    wines_df.at[index, "vinyard"] = vinyard
    wines_df.at[index, "variety"] = variety
    wines_df.at[index, "avi"] = avi
    wines_df.at[index, "region"] = region
    wines_df.at[index, "state"] = state
    wines_df.at[index, "alcohol_content"] = alcohol
    wines_df.at[index, "size"] = size
    wines_df.at[index, "winetype"] = winetype
    wines_df.at[index, "score"] = score   
    wines_df.at[index, "price"] = price
    wines_df.at[index, "dt_published"] = dt_published
    wines_df.at[index, "taster"] = taster
    
    print(index)
    index += 1

wines_df.head(5)

3648


,title,wine,vintage,vinyard,variety,avi,region,state,alcohol_content,size,winetype,price,score,dt_published,taster
954,Naggiar 2017 Estate Cabernet Sauvignon,Naggiar,2017,Estate,Cabernet Sauvignon,Sierra Foothills,Sierra Foothills,California,14.3%,750 ml,Red,$29,90,5/1/2019,Jim Gordon
955,Jeff Cohn 2016 Broken Compass Saitone Vineyard...,Jeff Cohn,2016,Broken Compass Saitone Vineyard Old Vine,"Carignane, Carignan",Russian River Valley,Sonoma,California,15.2%,750 ml,Red,$45,90,5/1/2019,Virginie Boone
956,La Crema 2015 Panorama Vineyard Pinot Noir,La Crema,2015,Panorama Vineyard,Pinot Noir,Arroyo Seco,Central Coast,California,14.5%,750 ml,Red,$40,90,5/1/2019,Matt Kettmann
957,Mt. Brave 2015 Cabernet Sauvignon,Mt. Brave,2015,Panorama Vineyard,Cabernet Sauvignon,Mount Veeder,Napa,California,14.5%,750 ml,Red,$90,90,5/1/2019,Virginie Boone
958,Naggiar 2017 Estate Cabernet Sauvignon,Naggiar,2017,Estate,Cabernet Sauvignon,Sierra Foothills,Sierra Foothills,California,14.3%,750 ml,Red,$29,90,5/1/2019,Jim Gordon


In [83]:
# trim white space around strings in df

def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if type(x) is str else x
    return df.applymap(trim_strings)

# fix current
wines_df = trim_all_columns(wines_df)

# save to csv
awfile_path = os.path.join("saveData","winesAll.csv")
wines_df.to_csv(awfile_path)

In [90]:
wines_df.tail()

,title,wine,vintage,vinyard,variety,avi,region,state,alcohol_content,size,winetype,price,score,dt_published,taster
3644,Maragas 2015 Chardonnay,Maragas,2015,1001,Chardonnay,Rattlesnake Hills,Columbia Valley,Washington,14.4%,750 ml,White,$39,81,2/1/2019,Sean P. Sullivan
3645,Schmidt 2016 Chardonnay,Schmidt,2016,1001,Chardonnay,Applegate Valley,Southern Oregon,Oregon,14.2%,750 ml,White,$26,81,2/1/2019,Paul Gregutt
3646,Schmidt 2016 Reserve Chardonnay,Schmidt,2016,Reserve,Chardonnay,Applegate Valley,Southern Oregon,Oregon,14.2%,750 ml,White,$24,81,2/1/2019,Paul Gregutt
3647,Wilridge 2017 Acadia Vineyard Pinot Grigio,Wilridge,2017,Acadia Vineyard,Pinot Grigio,Columbia Gorge (WA),Washington Other,Washington,13.4%,750 ml,White,$18,81,2/1/2019,Sean P. Sullivan
3648,Casa Larga 2017 Dry Riesling,Casa Larga,2017,Dry,Riesling,Finger Lakes,Finger Lakes,New York,12%,750 ml,White,$15,80,2/1/2019,Alexander Peartree


In [69]:
# check list length of wines_url_list
len(wines_url_list)

3649

In [71]:
# save wines_url_list to file

wineURLs = pd.DataFrame(wines_url_list, columns = ["url"])
wineURLs.head()
url_path = os.path.join("saveData","wineURLs.csv")
wineURLs.to_csv(url_path)